In [9]:
import math
import numpy as np

In [10]:
import random
from utils import procesar_dataframe, get_metricas

SCORES = [1, 2, 3, 4, 5]

ratings, test_ratings, NUM_ITEMS, NUM_USERS = procesar_dataframe('./DATOS/datasets_clase/All_Beauty.csv', 1, 5)

In [11]:
NUM_FACTORS = 7
LEARNING_RATE = 0.001
REGULARIZATION = 0.1

In [12]:
U = [[[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_USERS)] for _ in range(len(SCORES))]
V = [[[random.random() for _ in range(NUM_FACTORS)] for _ in range(NUM_ITEMS)] for _ in range(len(SCORES))]

In [13]:
def logit (x):
  return 1 / (1 + math.exp(-x))

In [14]:
def compute_prediction (u, i):
  prediction = None
  prob = 0
  for s in range(len(SCORES)):
    dot = np.dot(U[s][u], V[s][i])
    if logit(dot) > prob:
      prob = logit(dot)
      prediction = SCORES[s]
  return prediction, prob

## Aprendizaje de los factores latentes

El proceso de entrenamiento implicar aplicar las operaciones de actualización de las matrices de factores hasta que el algoritmo converja. En general, esta convergencia suele prefijarse como el número de iteraciones que realizamos sobre las operaciones de actualización:

In [15]:
NUM_ITERATIONS = 50

El siguiente código ejemplifica el proceso de entrenamiento del algoritmo:

In [16]:
for it in range(NUM_ITERATIONS):
  print("Iteración " + str(it + 1) + " de " + str(NUM_ITERATIONS))

  for s in range(len(SCORES)):

    # update users
    for u in range(NUM_USERS):
      delta = np.zeros(NUM_FACTORS)
      for i in range(NUM_ITEMS):
        if ratings[u][i] != None:
          dot = np.dot(U[s][u], V[s][i])

          for f in range(NUM_FACTORS):
            if ratings[u][i] == SCORES[s]:
              delta[f] += (1 - logit(dot)) * V[s][i][f]
            else:
              delta[f] -= logit(dot) * V[s][i][f]

          for f in range(NUM_FACTORS):
            U[s][u][f] += LEARNING_RATE * (delta[f] - REGULARIZATION * U[s][u][f])

    # update items
    for i in range(NUM_ITEMS):
      theta = np.zeros(NUM_FACTORS)
      for u in range(NUM_USERS):
        if ratings[u][i] != None:
          dot = np.dot(U[s][u], V[s][i])

          for f in range(NUM_FACTORS):
            if ratings[u][i] == SCORES[s]:
              theta[f] += (1 - logit(dot)) * U[s][u][f]
            else:
              theta[f] -= logit(dot) * U[s][u][f]

          for f in range(NUM_FACTORS):
            V[s][i][f] += LEARNING_RATE * (theta[f] - REGULARIZATION * V[s][i][f])

Iteración 1 de 50
Iteración 2 de 50
Iteración 3 de 50
Iteración 4 de 50
Iteración 5 de 50
Iteración 6 de 50
Iteración 7 de 50
Iteración 8 de 50
Iteración 9 de 50
Iteración 10 de 50
Iteración 11 de 50
Iteración 12 de 50
Iteración 13 de 50
Iteración 14 de 50
Iteración 15 de 50
Iteración 16 de 50
Iteración 17 de 50
Iteración 18 de 50
Iteración 19 de 50
Iteración 20 de 50
Iteración 21 de 50
Iteración 22 de 50
Iteración 23 de 50
Iteración 24 de 50
Iteración 25 de 50
Iteración 26 de 50
Iteración 27 de 50
Iteración 28 de 50
Iteración 29 de 50
Iteración 30 de 50
Iteración 31 de 50
Iteración 32 de 50
Iteración 33 de 50
Iteración 34 de 50
Iteración 35 de 50
Iteración 36 de 50
Iteración 37 de 50
Iteración 38 de 50
Iteración 39 de 50
Iteración 40 de 50
Iteración 41 de 50
Iteración 42 de 50
Iteración 43 de 50
Iteración 44 de 50
Iteración 45 de 50
Iteración 46 de 50
Iteración 47 de 50
Iteración 48 de 50
Iteración 49 de 50
Iteración 50 de 50


## Cálculo de las recomendaciones

El cálculo de las recomendaciones, por lo general, simplemente implica seleccionar los *N* items con una predicción más alta. Por ejemplo, si quisiéramos recomendar *N = 3* items a un usuario que tuviera las siguientes predicciones:

|   	| i1 	| i2 	| i3 	| i4 	| i5 	| i6 	| i7 	| i8 	| i9 	| i10 	|
|:-:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|:--:	|-----	|
| u 	|   	|  2,9 	|    	|  4,7 	|  5,0 	|    	|  1,2 	|    	|   	|  3,1 	|

Se le recomendarían a dicho usuario los items *i5*, *i4* e *i10*.

In [17]:
N = 5

In [18]:
predictions = [[None for _ in range(NUM_ITEMS)] for _ in range(NUM_USERS)]

# Rellenamos la matriz de predicciones
for u in range(NUM_USERS):
  for i in range(NUM_ITEMS):
    if test_ratings[u][i] != None:
      pred, prob = compute_prediction(u, i)
      predictions[u][i] = pred

In [19]:
get_metricas(test_ratings, NUM_ITEMS, NUM_USERS, predictions, 4, 1)

MAE =  0.2486521164021164
RMSE =  0.3718824580060556
Precision =  0.9666666666666667
Recall =  0.4052372962607857
F1 =  0.5559502019233563


## Referencias

Ortega, F., Lara-Cabrera, R., González-Prieto, Á., & Bobadilla, J. (2021). **Providing reliability in recommender systems through Bernoulli Matrix Factorization**. Information Sciences, 553, 110-128.
